In [3]:
import sys
sys.path.insert(0, '..')
import os
import glob
from filament import filament
import datetime
import cmocean
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [7]:
datadir = "/data/SST/SouthAfrica/"
figdir = "/data/SST/SouthAfrica/figures"
visiblefile = "/data/Visible/VIIRS_20170902.tiff"
datafilelist = sorted(glob.glob(os.path.join(datadir, "*SST.nc")))
if not(os.path.exists(figdir)):
    os.makedirs(figdir)

In [6]:
visibleim = filament.Visible()
visibleim.read_geotiff(visiblefile)

In [8]:
coordinates = (7., 19., -35., -23.)

In [9]:
m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='i')

In [21]:
def make_plot_sst(sst, figname):
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot(111)

    # Background: visible image grey
    m.pcolormesh(visibleim.lon, visibleim.lat, visibleim.image[:,:,0], latlon=True, 
             cmap=plt.cm.gray, zorder=1)
    pcm = m.pcolormesh(sst.lon, sst.lat, sst.field, latlon=True,
                 cmap=cmocean.cm.thermal, vmin=11., vmax=18., zorder=2)
    cb = plt.colorbar(pcm, extend="both")
    cb.set_label("$^{\circ}$C", rotation=0, ha="left")
    m.drawcoastlines(linewidth=0.25)
    #m.fillcontinents(zorder=4)
    m.drawmeridians(np.arange(coordinates[0], coordinates[1], 3.), labels=(0,0,1,0), 
                    linewidth=.5, fontsize=14, zorder=3)
    m.drawparallels(np.arange(coordinates[2], coordinates[3], 2.), labels=(1,0,0,0), 
                    linewidth=.5, fontsize=14, zorder=3)
    plt.savefig(figname, dpi=300, bbox_inches="tight")
    # plt.show()
    plt.close()

In [22]:
for datafile in datafilelist[0:1]:
    sst = filament.SST()
    sst.read_from_oceancolorL2(datafile)
    sst.field = np.ma.masked_where(sst.qflag > 1, sst.field)
    titletext = sst.get_title()
    figname = os.path.join(figdir, sst.get_figname())
    make_plot_sst(sst, figname)